# Uber H3 Demo
H3 is hierarchical spatial index solution that applies recursive decomposition based on hexagonal cells.



## Setup
Need to install h3 and folium first

In [177]:
%pip install h3
%pip install folium
%pip install geopandas
%pip install tabulate

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## Helper Functions
This section will walk reader thorough the helper functions.

Let's first import packages this notebook will need.

In [163]:
import h3
import folium
from shapely.geometry import MultiPolygon, Polygon
from IPython.display import display
import pandas as pd

Let's display the H3 Cell id of a location at different hexagon resolutions.

In [172]:
# Let's explore H3Cell
df = pd.DataFrame(columns=["Lat", "Lon", "Hexagon Resolution", "H3 Address", "H3 Cell First Part", "H3 Cell Second Part"])
test_lat = 37.3615593
test_lon = -122.0553238
test_hexagon_resolutions = [7,8,9,10,11,12,13,14,15]
for i in test_hexagon_resolutions:
    hexagon_resolution=i
    h3_address = h3.latlng_to_cell(test_lat,test_lon, hexagon_resolution) # lat, lng, hex resolution
    print(f"H3 address ({test_lat},{test_lon},{hexagon_resolution:02d}): {h3_address} -> {h3_address[:3]} {h3_address[3:]}")
    new_row = {"Lat":test_lat, "Lon":test_lon, "Hexagon Resolution": hexagon_resolution, "H3 Address": h3_address, "H3 Cell First Part": h3_address[:3], "H3 Cell Second Part": h3_address[3:] }
    df.loc[len(df)]=new_row


H3 address (37.3615593,-122.0553238,07): 87283472bffffff -> 872 83472bffffff
H3 address (37.3615593,-122.0553238,08): 88283472b3fffff -> 882 83472b3fffff
H3 address (37.3615593,-122.0553238,09): 89283470d93ffff -> 892 83470d93ffff
H3 address (37.3615593,-122.0553238,10): 8a283470d927fff -> 8a2 83470d927fff
H3 address (37.3615593,-122.0553238,11): 8b283470d921fff -> 8b2 83470d921fff
H3 address (37.3615593,-122.0553238,12): 8c283470d921dff -> 8c2 83470d921dff
H3 address (37.3615593,-122.0553238,13): 8d283470d921c7f -> 8d2 83470d921c7f
H3 address (37.3615593,-122.0553238,14): 8e283470d921c67 -> 8e2 83470d921c67
H3 address (37.3615593,-122.0553238,15): 8f283470d921c65 -> 8f2 83470d921c65


When the H3 Cell id of the same location at different resolution is displayed, it is easier to recognize the hierarchical spatial index encoding. For instance, the second character correlates with the resolution. In the last 12 characters, it is easier to observe that the prefix emerges as the resolution is increased.

In [176]:
from tabulate import tabulate
print(tabulate(df, headers = 'keys', tablefmt = 'psql'))

+----+---------+----------+----------------------+-----------------+----------------------+-----------------------+
|    |     Lat |      Lon |   Hexagon Resolution | H3 Address      | H3 Cell First Part   | H3 Cell Second Part   |
|----+---------+----------+----------------------+-----------------+----------------------+-----------------------|
|  0 | 37.3616 | -122.055 |                    7 | 87283472bffffff | 872                  | 83472bffffff          |
|  1 | 37.3616 | -122.055 |                    8 | 88283472b3fffff | 882                  | 83472b3fffff          |
|  2 | 37.3616 | -122.055 |                    9 | 89283470d93ffff | 892                  | 83470d93ffff          |
|  3 | 37.3616 | -122.055 |                   10 | 8a283470d927fff | 8a2                  | 83470d927fff          |
|  4 | 37.3616 | -122.055 |                   11 | 8b283470d921fff | 8b2                  | 83470d921fff          |
|  5 | 37.3616 | -122.055 |                   12 | 8c283470d921dff | 8c2

In [131]:
# Extract the vertices from MultiPolygon structure, assuming 1 polygon
def get_outline_coordinates(mpoly: MultiPolygon):
    outline_coordinates = []
    for polygon in mpoly['coordinates']:
        if polygon:
            for vertices in polygon:
                if vertices:
                    for coord in vertices:
                        outline_coordinates.append([coord[0],coord[1]]) ## needs further investigation why it needs to be switched
    return outline_coordinates

In [69]:
# Defines line segments from list of vertices
# assumption: vertices are in order
def get_polyLines(coordinates):
    returnlines = []
    i=1;
    while i < len(coordinates):
        returnlines.append([coordinates[i-1],coordinates[i]])
        i=i+1
    return returnlines

In [145]:
# Switching
def reverseLatLonLinesOfPolygon(vertices):
    result = []
    [result.append([v[1], v[0]]) for v in vertices]
    return result

Method visualize_polygon() renders polygon a map (folium.Map). <br>
The returned map can be rendered by calling display(m)

In [44]:
###
def visualize_polygon(polyline, color):
    """
        Visualize a polygon by using folium
    :param polyline:
    :param color:
    :return:
    """
    polyline.append(polyline[0])
    lat = [p[0] for p in polyline]
    lng = [p[1] for p in polyline]
    m = folium.Map(location=[sum(lat)/len(lat), sum(lng)/len(lng)], zoom_start=13, tiles='Cartodb Positron')
    my_PolyLine=folium.PolyLine(locations=polyline,weight=8,color=color)
    m.add_child(my_PolyLine)
    return m


Method visualize_hexagons() renders hexagons on a map (folium.Map)

In [153]:
###
def visualize_hexagons(hexagons, color="cyan", folium_map=None):
    """
    Visualize Hexagons by using folium tool

    :param hexagons: is a list of hexcluster. Each hexcluster is a list of hexagons. eg. [[hex1, hex2], [hex3, hex4]]
    :param color:
    :param folium_map:
    """
    polylines = []
    lat = []
    lng = []
    for hex in hexagons:
        #polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)
        polygons = h3.cells_to_geo([hex], tight=False)
        # flatten polygons into loops.
        #outlines = [loop for polygon in polygons for loop in polygon]
        outlines = get_outline_coordinates(polygons)
        #polyline = [outline + [outline[0]] for outline in outlines][0]
        polyline = get_polyLines(outlines)
        #polyline = h3.LatLngPoly(outlines)
        polyline = outlines
        lat.extend(map(lambda v:v[0],polyline))
        lng.extend(map(lambda v:v[1],polyline))
        polylines.append(polyline)

    #for polyline in polylines:
    #    print(polyline)
    location=[sum(lat)/len(lat), sum(lng)/len(lng)]
    print(f"Center :{location}")

    if folium_map is None:
        #m = folium.Map(location=[sum(lng)/len(lng), sum(lat)/len(lat)], tiles='Cartodb Positron', zoom_start=13)
        m = folium.Map(location=[sum(lng)/len(lng), sum(lat)/len(lat)], tiles='OpenStreetMap', zoom_start=13)
    else:
        m = folium_map

    for polyline in polylines:
        #my_PolyLine=folium.PolyLine(locations=polyline,weight=8,color=color)
        my_PolyLine=folium.PolyLine(locations=reverseLatLonLinesOfPolygon(polyline),weight=8,color=color)
        m.add_child(my_PolyLine)
        for v in polyline:
            r=5
            circle = folium.CircleMarker(
                location=[v[1], v[0]],
                radius=r,
                color="cornflowerblue",
                stroke=False,
                fill=True,
                fill_opacity=0.6,
                opacity=1,
                popup="{} pixels".format(r),
                tooltip="I am in pixels",
            )
            m.add_child(circle)

    return m


## H3 Cell Visualization on a Map
- It will translate location at resolution 9 to H3 Cell address
- Then finds the vertices of H3 Cell
- Displays the boundary of H3 Cell on a Map

In [180]:
hexagon_resolution=9
h3_address = h3.latlng_to_cell(test_lat,test_lon, hexagon_resolution)
print(f"h3.latlng_to_cell ({test_lat},{test_lon},{hexagon_resolution:02d}): {h3_address} ")


h3.latlng_to_cell (37.3615593,-122.0553238,09): 89283470d93ffff 


As we have seen before H3 Cell did consist of two parts: first 3 encodes and the remaining encodes the hierarchy.
<br> Let's find out the boundary of 89283470d93ffff

In [183]:
# Boundary of H3 Cell
h3_cell_boundary = h3.cell_to_boundary(h3_address)
for i in range(len(h3_cell_boundary)):
    print(f"v[{i+1}]={h3_cell_boundary[i]}")

v[1]=(37.3611559268098, -122.05318357743447)
v[2]=(37.36284119732859, -122.05210810145715)
v[3]=(37.3643578433723, -122.05336039429898)
v[4]=(37.36418919531743, -122.05568816325125)
v[5]=(37.36250392408233, -122.05676357113303)
v[6]=(37.36098730161813, -122.05551127816072)


Because this H3 Cell is hexagon, it has 6 vertices. <br> Let's display H3 Cell information on a map.

In [187]:
m = visualize_hexagons([h3_address],color="red")
display(m)

Center :[-122.05425695188144, 37.36245590219119]


Let's also visualize the parent H3 Cell

In [184]:
parents = h3.cell_to_parent(h3_address)
print(f"parent-of({h3_address})={parents}")

parent-of(89283470d93ffff)=88283470d9fffff


In [188]:
m = visualize_hexagons([parents],color="blue",folium_map=m)
display(m)

Center :[-122.05545664103182, 37.366042782302706]


## Summary
The objective of this exploration is to demonstrate part of H3 Cell and spatial indexing. Hopefully it helps to understand how it works.